## 데이터 전처리
### 기본 설정

In [1]:
# 패키지 로드
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from collections import defaultdict
import os, random

from scipy import sparse
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
from torch.nn.init import normal_
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

import plotnine
from plotnine import *

In [2]:
# 경로 설정
data_path = './data'
saved_path = './code/saved'
output_path = './code/submission'

In [3]:
# 데이터 불러오기 
history_df = pd.read_csv(os.path.join(data_path, 'history_data.csv'), encoding='utf-8')
search_df = pd.read_csv(os.path.join(data_path, 'search_data.csv'), encoding='utf-8')
profile_df = pd.read_csv(os.path.join(data_path, 'profile_data.csv'), encoding='utf-8')
meta_df = pd.read_csv(os.path.join(data_path, 'meta_data.csv'), encoding='utf-8')
we_df = pd.read_csv(os.path.join(data_path, 'watch_e_data.csv'), encoding='utf-8')
buy_df = pd.read_csv(os.path.join(data_path, 'buy_data.csv'), encoding='utf-8')

In [ ]:
history_df.drop_duplicates(['profile_id', 'album_id'])

In [242]:
history_df.groupby(['profile_id', 'album_id', 'short_trailer']).count()

ss_id  log_time  act_target_dtl  payment  \
profile_id album_id short_trailer                                             
3          15       N                  1         1               1        0   
           16       N                  1         1               1        0   
           17       N                  1         1               1        0   
           18       N                  1         1               1        0   
           19       N                  1         1               1        0   
...                                  ...       ...             ...      ...   
33032      7105     N                 21        21              21        0   
           8910     N                  1         1               1        0   
           14662    Y                  1         1               1        0   
           14663    Y                  2         2               2        0   
           14665    Y                  1         1               1        0   

                                   continuous_play  
profile_id album_id short_trailer                   
3          15       N                            1  
           16       N                            1  
           17       N                            1  
           18       N                            1  
           19       N                            1  
...                                            ...  
33032      7105     N                           21  
           8910     N                            1  
           14662    Y                            1  
           14663    Y                            2  
           14665    Y                            1  

[381161 rows x 5 columns]

In [231]:
history_df.loc[history_df['short_trailer'] == 'Y']

,profile_id,ss_id,log_time,act_target_dtl,album_id,payment,continuous_play,short_trailer
20,3,20220301115653,20220301123505,MKID003,33,NaN,Y,Y
123,5,20220309185728,20220309195543,MKID003,138,NaN,Y,Y
144,5,20220313130615,20220313131521,MKID003,150,NaN,Y,Y
156,5,20220318212718,20220318212809,MKID003,159,NaN,Y,Y
157,5,20220318212718,20220318212811,MKID003,160,NaN,Y,Y
...,...,...,...,...,...,...,...,...
1005593,33032,20220427112116,20220427112335,MKID003,14662,NaN,Y,Y
1005594,33032,20220427112116,20220427112369,MKID003,14663,NaN,Y,Y
1005595,33032,20220427112116,20220427112384,MKID003,14663,NaN,Y,Y
1005596,33032,20220427112116,20220427112394,MKID003,7010,NaN,Y,Y


In [268]:
short = history_df[['profile_id', 'album_id', 'short_trailer']].drop_duplicates(['profile_id', 'album_id', 'short_trailer'])

In [269]:
short

,profile_id,album_id,short_trailer
0,3,15,N
1,3,16,N
2,3,17,N
3,3,18,N
4,3,19,N
...,...,...,...
1005610,33032,2133,N
1005616,33032,190,N
1005623,33032,381,N
1005637,33032,327,N


In [272]:
# 예고편을 시청한 사용자 -> 18047건
short.drop_duplicates(['profile_id', 'album_id']).loc[short['short_trailer'] == 'Y']

# 그 중 612건만이 실제 시청으로 이어짐?

,profile_id,album_id,short_trailer
20,3,33,Y
123,5,138,Y
144,5,150,Y
156,5,159,Y
157,5,160,Y
...,...,...,...
1005592,33032,14665,Y
1005593,33032,14662,Y
1005594,33032,14663,Y
1005596,33032,7010,Y


In [ ]:
short.pro

In [10]:
history_df.payment.describe()

count    73486.000000
mean      2117.477207
std       3612.951558
min        300.000000
25%        500.000000
50%        500.000000
75%       1400.000000
max      25000.000000
Name: payment, dtype: float64

In [6]:
buy_df

,profile_id,ss_id,log_time,act_target_dtl,album_id,payment
0,255,20220426153942,20220426154128,MKID004,3767,500
1,1603,20220309163919,20220309164043,MKID004,1640,300
2,5303,20220411080324,20220411082725,MKID004,3821,500
3,5303,20220411080324,20220411084013,MKID004,3822,500
4,5303,20220411080324,20220411085607,MKID004,3823,500
...,...,...,...,...,...,...
145,23882,20220421182514,20220421185231,MKID004,7441,300
146,23882,20220421182514,20220421185510,MKID004,8749,1200
147,31509,20220409115246,20220409115347,MKID004,19906,500
148,31837,20220427143436,20220427143618,MKID004,1640,300


In [230]:
history_df

,profile_id,ss_id,log_time,act_target_dtl,album_id,payment,continuous_play,short_trailer
0,3,20220301115653,20220301115719,MKID003,15,NaN,Y,N
1,3,20220301115653,20220301115809,MKID003,16,NaN,Y,N
2,3,20220301115653,20220301115958,MKID003,17,NaN,Y,N
3,3,20220301115653,20220301120118,MKID003,18,NaN,Y,N
4,3,20220301115653,20220301120229,MKID003,19,NaN,Y,N
...,...,...,...,...,...,...,...,...
1005646,33032,20220427155091,20220427155668,MKID003,381,NaN,Y,N
1005647,33032,20220427155091,20220427155680,MKID003,381,NaN,Y,N
1005648,33032,20220427155839,20220427155810,MKID003,125,NaN,Y,N
1005649,33032,20220427155706,20220427155838,MKID003,125,NaN,Y,N


In [251]:
history_df.loc[history_df['short_trailer'] == 'Y']

,profile_id,ss_id,log_time,act_target_dtl,album_id,payment,continuous_play,short_trailer
20,3,20220301115653,20220301123505,MKID003,33,NaN,Y,Y
123,5,20220309185728,20220309195543,MKID003,138,NaN,Y,Y
144,5,20220313130615,20220313131521,MKID003,150,NaN,Y,Y
156,5,20220318212718,20220318212809,MKID003,159,NaN,Y,Y
157,5,20220318212718,20220318212811,MKID003,160,NaN,Y,Y
...,...,...,...,...,...,...,...,...
1005593,33032,20220427112116,20220427112335,MKID003,14662,NaN,Y,Y
1005594,33032,20220427112116,20220427112369,MKID003,14663,NaN,Y,Y
1005595,33032,20220427112116,20220427112384,MKID003,14663,NaN,Y,Y
1005596,33032,20220427112116,20220427112394,MKID003,7010,NaN,Y,Y


In [261]:
a = history_df.groupby(['profile_id', 'album_id']).nunique().short_trailer.reset_index()

In [264]:
# 같은 사용자가 어떤 앨범을 예고편을 보고 본편을 봤다고 이해할 수 있는 경우 -> 614건..
a.loc[a['short_trailer'] == 2]

,profile_id,album_id,short_trailer
1702,101,1998,2
1792,103,124,2
1983,121,3001,2
1984,121,3002,2
2770,156,175,2
...,...,...,...
377461,32595,175,2
378493,32725,442,2
379188,32791,478,2
379199,32791,2491,2


In [265]:
history_df.groupby(['profile_id', 'album_id']).cou

In [243]:
we_df['total_time'].describe()

count    892794.000000
mean        338.502926
std         346.333053
min          19.000000
25%         104.000000
50%         271.000000
75%         478.000000
max        8929.000000
Name: total_time, dtype: float64

In [250]:
meta_df.loc[meta_df['album_id']== 442]

,album_id,title,sub_title,genre_large,genre_mid,genre_small,country,run_time,onair_date,cast_1,cast_2,cast_3,cast_4,cast_5,cast_6,cast_7
40575,442,방아 찧는 호랑이,4-5세,키즈,책,NaN,한국,390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [244]:
we_df.loc[we_df['total_time'] == 19]

,profile_id,ss_id,log_time,act_target_dtl,album_id,watch_time,total_time,continuous_play
75578,2367,20220318081787,20220318101108,MKID049,2559,19,19,1
75858,2367,20220322193339,20220322194337,MKID049,2559,19,19,1
75910,2367,20220323092577,20220323094386,MKID049,2559,19,19,1
75911,2367,20220323092577,20220323094386,MKID049,2559,19,19,1
75912,2367,20220323092577,20220323094386,MKID049,2559,19,19,1
75913,2367,20220323092577,20220323094386,MKID049,2559,19,19,1
76088,2367,20220326084119,20220326123925,MKID049,2559,19,19,1
79135,2447,20220310073943,20220311064690,MKID049,2559,19,19,1
79136,2447,20220310073943,20220311064690,MKID049,2559,19,19,1
79137,2447,20220310073943,20220311064690,MKID049,2559,19,19,1


In [8]:
buy_df.shape

(150, 6)

In [7]:
buy_df.payment.describe()

count     150.000000
mean      706.000000
std       843.469186
min       300.000000
25%       300.000000
50%       500.000000
75%       500.000000
max      6300.000000
Name: payment, dtype: float64

## 장르와 키워드 사이의 관계 파악
- history_df에 앨범별 장르와 사용자별 부모키워드123, 아이키워드 123 병합

In [51]:
### 키워드 별 인원을 나타내기위해 하나로 모으기
keyword = profile_df[['profile_id',
                      'pr_interest_keyword_cd_1', 'pr_interest_keyword_cd_2', 'pr_interest_keyword_cd_3',
                      'ch_interest_keyword_cd_1', 'ch_interest_keyword_cd_2', 'ch_interest_keyword_cd_3']]

In [49]:
genre = meta_df[['album_id', 'genre_mid']].drop_duplicates('album_id')

In [50]:
genre

,album_id,genre_mid
0,749,TV만화
1,750,TV만화
2,2131,TV만화
3,2625,TV만화
4,2594,TV만화
...,...,...
42596,39872,놀이교실
42597,39873,놀이교실
42598,39874,놀이교실
42599,4779,책


In [52]:
keyword

,profile_id,pr_interest_keyword_cd_1,pr_interest_keyword_cd_2,pr_interest_keyword_cd_3,ch_interest_keyword_cd_1,ch_interest_keyword_cd_2,ch_interest_keyword_cd_3
0,3,P02,P04,P07,K01,K03,K04
1,5,P07,P08,P06,K05,K08,K09
2,7,P05,P03,NaN,K06,K04,NaN
3,12,P03,P06,P02,K09,K07,K03
4,16,P03,P06,P01,K01,K06,K04
...,...,...,...,...,...,...,...
8306,33022,P04,NaN,NaN,K04,K08,NaN
8307,33023,P06,P03,P07,K08,K04,K05
8308,33026,P01,P03,P08,K05,K09,K06
8309,33027,P04,P05,P06,K03,K01,K05


In [56]:
data = history_df.drop_duplicates(['profile_id', 'album_id'])[['profile_id', 'album_id']]

In [57]:
data

,profile_id,album_id
0,3,15
1,3,16
2,3,17
3,3,18
4,3,19
...,...,...
1005610,33032,2133
1005616,33032,190
1005623,33032,381
1005637,33032,327


In [63]:
### data에 키워드 merge
k = pd.merge(data, keyword, on = 'profile_id')
new = pd.merge(k, genre, on = 'album_id')

In [64]:
new

,profile_id,album_id,pr_interest_keyword_cd_1,pr_interest_keyword_cd_2,pr_interest_keyword_cd_3,ch_interest_keyword_cd_1,ch_interest_keyword_cd_2,ch_interest_keyword_cd_3,genre_mid
0,3,15,P02,P04,P07,K01,K03,K04,노래율동
1,5,15,P07,P08,P06,K05,K08,K09,노래율동
2,22,15,P06,NaN,NaN,K01,NaN,NaN,노래율동
3,96,15,P06,P07,P04,K08,K03,K01,노래율동
4,101,15,P06,P07,P02,K01,K08,K02,노래율동
...,...,...,...,...,...,...,...,...,...
380542,32965,25913,P01,NaN,NaN,K01,K08,NaN,노래율동
380543,32965,17289,P01,NaN,NaN,K01,K08,NaN,노래율동
380544,32965,25914,P01,NaN,NaN,K01,K08,NaN,노래율동
380545,32965,25915,P01,NaN,NaN,K01,K08,NaN,노래율동


In [78]:
cross.columns

Index(['P01', 'P02', 'P03', 'P04', 'P05', 'P06', 'P07', 'P08'], dtype='object', name='pr_interest_keyword_cd_1')

In [87]:
pr_key_dict = {'P01': '과학기술', 'P02': '정서/사회성', 'P03': '자연탐구', 'P04': '바른생활/안전',
               'P05': '활동/운동', 'P06': '음악예술', 'P07': '언어논리', 'P08': '수리논리'}

func = lambda x: pr_key_dict.get(x, x)

new['pr_key1'] = new.pr_interest_keyword_cd_1.map(func)
new['pr_key2'] = new.pr_interest_keyword_cd_2.map(func)
new['pr_key3'] = new.pr_interest_keyword_cd_3.map(func)

In [84]:
ch_key_dict = {'K01': '노래/율동', 'K02': '동물/식물', 'K03': '동화', 'K04': '만들기',
               'K05': '숫자/계산', 'K06': '외국어', 'K07': '친구/사람', 'K08': '탈것/기계', 'K09': '활동/운동'}

func1 = lambda x: ch_key_dict.get(x, x)

new['ch_key1'] = new.ch_interest_keyword_cd_1.map(func1)
new['ch_key2'] = new.ch_interest_keyword_cd_2.map(func1)
new['ch_key3'] = new.ch_interest_keyword_cd_3.map(func1)

In [89]:
new = new[['profile_id', 'album_id', 'genre_mid', 'pr_key1', 'pr_key2', 'pr_key3', 'ch_key1', 'ch_key2', 'ch_key3']]
new

,profile_id,album_id,genre_mid,pr_key1,pr_key2,pr_key3,ch_key1,ch_key2,ch_key3
0,3,15,노래율동,정서/사회성,바른생활/안전,언어논리,노래/율동,동화,만들기
1,5,15,노래율동,언어논리,수리논리,음악예술,숫자/계산,탈것/기계,활동/운동
2,22,15,노래율동,음악예술,NaN,NaN,노래/율동,NaN,NaN
3,96,15,노래율동,음악예술,언어논리,바른생활/안전,탈것/기계,동화,노래/율동
4,101,15,노래율동,음악예술,언어논리,정서/사회성,노래/율동,탈것/기계,동물/식물
...,...,...,...,...,...,...,...,...,...
380542,32965,25913,노래율동,과학기술,NaN,NaN,노래/율동,탈것/기계,NaN
380543,32965,17289,노래율동,과학기술,NaN,NaN,노래/율동,탈것/기계,NaN
380544,32965,25914,노래율동,과학기술,NaN,NaN,노래/율동,탈것/기계,NaN
380545,32965,25915,노래율동,과학기술,NaN,NaN,노래/율동,탈것/기계,NaN


In [204]:
user_df = profile_df
user_df.columns = ['profile_id', 'sex', 'age', 'pr_key1', 'pr_key2', 'pr_key3', 'ch_key1', 'ch_key2', 'ch_key3']

pr_key_dict = {'P01': '과학기술', 'P02': '정서/사회성', 'P03': '자연탐구', 'P04': '바른생활/안전',
               'P05': '활동/운동', 'P06': '음악예술', 'P07': '언어논리', 'P08': '수리논리'}

func = lambda x: pr_key_dict.get(x, x)

user_df['pr_key1'] = user_df.pr_key1.map(func)
user_df['pr_key2'] = user_df.pr_key2.map(func)
user_df['pr_key3'] = user_df.pr_key3.map(func)


ch_key_dict = {'K01': '노래/율동', 'K02': '동물/식물', 'K03': '동화', 'K04': '만들기',
               'K05': '숫자/계산', 'K06': '외국어', 'K07': '친구/사람', 'K08': '탈것/기계', 'K09': '활동/운동'}

func1 = lambda x: ch_key_dict.get(x, x)

user_df['ch_key1'] = user_df.ch_key1.map(func1)
user_df['ch_key2'] = user_df.ch_key2.map(func1)
user_df['ch_key3'] = user_df.ch_key3.map(func1)

user_df = user_df.fillna('nan')
user_df

,profile_id,sex,age,pr_key1,pr_key2,pr_key3,ch_key1,ch_key2,ch_key3
0,3,F,5,정서/사회성,바른생활/안전,언어논리,노래/율동,동화,만들기
1,5,M,5,언어논리,수리논리,음악예술,숫자/계산,탈것/기계,활동/운동
2,7,F,9,활동/운동,자연탐구,nan,외국어,만들기,nan
3,12,M,6,자연탐구,음악예술,정서/사회성,활동/운동,친구/사람,동화
4,16,F,12,자연탐구,음악예술,과학기술,노래/율동,외국어,만들기
...,...,...,...,...,...,...,...,...,...
8306,33022,M,1,바른생활/안전,nan,nan,만들기,탈것/기계,nan
8307,33023,M,5,음악예술,자연탐구,언어논리,탈것/기계,만들기,숫자/계산
8308,33026,F,8,과학기술,자연탐구,수리논리,숫자/계산,활동/운동,외국어
8309,33027,F,4,바른생활/안전,활동/운동,음악예술,동화,노래/율동,숫자/계산


In [189]:
pr_key_list = ['과학기술', '정서/사회성', '자연탐구', '바른생활/안전', '활동/운동', '음악예술', '언어논리', '수리논리']
ch_key_list = ['노래/율동', '동물/식물', '동화', '만들기', '숫자/계산', '외국어', '친구/사람', '탈것/기계', '활동/운동']

In [198]:
pr_dict = dict()
i = 0
for a, b in combinations(pr_key_list, 2):
    pr_dict[(a, b)] = i
    i += 1

for a in pr_key_list:
    pr_dict[(a, 'nan')] = i
    i += 1

ch_dict = dict()
i = 0
for a, b in combinations(ch_key_list, 2):
    ch_dict[(a, b)] = i
    i += 1

for a in ch_key_list:
    ch_dict[(a, 'nan')] = i
    i += 1

{('과학기술', '정서/사회성'): 0,
 ('과학기술', '자연탐구'): 1,
 ('과학기술', '바른생활/안전'): 2,
 ('과학기술', '활동/운동'): 3,
 ('과학기술', '음악예술'): 4,
 ('과학기술', '언어논리'): 5,
 ('과학기술', '수리논리'): 6,
 ('정서/사회성', '자연탐구'): 7,
 ('정서/사회성', '바른생활/안전'): 8,
 ('정서/사회성', '활동/운동'): 9,
 ('정서/사회성', '음악예술'): 10,
 ('정서/사회성', '언어논리'): 11,
 ('정서/사회성', '수리논리'): 12,
 ('자연탐구', '바른생활/안전'): 13,
 ('자연탐구', '활동/운동'): 14,
 ('자연탐구', '음악예술'): 15,
 ('자연탐구', '언어논리'): 16,
 ('자연탐구', '수리논리'): 17,
 ('바른생활/안전', '활동/운동'): 18,
 ('바른생활/안전', '음악예술'): 19,
 ('바른생활/안전', '언어논리'): 20,
 ('바른생활/안전', '수리논리'): 21,
 ('활동/운동', '음악예술'): 22,
 ('활동/운동', '언어논리'): 23,
 ('활동/운동', '수리논리'): 24,
 ('음악예술', '언어논리'): 25,
 ('음악예술', '수리논리'): 26,
 ('언어논리', '수리논리'): 27,
 ('과학기술', 'nan'): 28,
 ('정서/사회성', 'nan'): 29,
 ('자연탐구', 'nan'): 30,
 ('바른생활/안전', 'nan'): 31,
 ('활동/운동', 'nan'): 32,
 ('음악예술', 'nan'): 33,
 ('언어논리', 'nan'): 34,
 ('수리논리', 'nan'): 35}

In [216]:
user_df['pr_key_tuple'] = np.nan
user_df['ch_key_tuple'] = np.nan

for i in user_df.itertuples():
    if (i[4], i[5]) in pr_dict.keys():
        user_df['pr_key_tuple'][i[0]] = pr_dict[(i[4], i[5])]
    else:
        user_df['pr_key_tuple'][i[0]] = pr_dict[(i[5], i[4])]

for i in user_df.itertuples():
    if (i[7], i[8]) in ch_dict.keys():
        user_df['ch_key_tuple'][i[0]] = ch_dict[(i[7], i[8])]
    else:
        user_df['ch_key_tuple'][i[0]] = ch_dict[(i[8], i[7])]

user_df['ch_key_tuple'] = le.fit_transform(user_df['ch_key_tuple'])
user_df['pr_key_tuple'] = le.fit_transform(user_df['pr_key_tuple'])

In [227]:
user_df = user_df[['profile_id' ,'sex', 'age', 'pr_key_tuple', 'ch_key_tuple']]
user_df

,profile_id,sex,age,pr_key_tuple,ch_key_tuple
0,3,F,5,8,1
1,5,M,5,27,28
2,7,F,9,14,22
3,12,M,6,15,34
4,16,F,12,15,4
...,...,...,...,...,...
8306,33022,M,1,31,24
8307,33023,M,5,15,24
8308,33026,F,8,1,29
8309,33027,F,4,18,1


In [224]:
user_df['ch_key_tuple'].value_counts()

0     1166
1      816
36     649
8      332
15     317
2      304
4      291
9      271
6      270
17     265
16     225
38     222
26     204
37     194
3      191
13     168
39     165
21     165
43     164
7      154
5      141
35     110
11     104
19     103
22     102
44      99
10      97
41      89
25      82
14      81
28      75
31      75
23      73
12      71
18      64
40      60
24      59
29      52
42      51
20      46
30      35
32      29
34      28
33      27
27      25
Name: ch_key_tuple, dtype: int64

In [225]:
user_df['pr_key_tuple'].value_counts()

25    516
10    515
7     493
33    463
15    445
1     423
27    411
22    379
8     375
11    352
4     297
0     290
32    227
34    214
16    208
5     207
13    195
9     195
19    190
14    188
28    171
29    169
20    164
30    152
23    148
3     142
6     140
18    119
26    106
31     96
2      80
12     64
17     56
21     43
35     41
24     37
Name: pr_key_tuple, dtype: int64

In [113]:
# 38만 건 모두
new['ch_key1'].value_counts()

노래/율동    176223
동물/식물     50879
동화        44724
만들기       27848
탈것/기계     24475
외국어       22288
숫자/계산     16781
활동/운동     11230
친구/사람      6099
Name: ch_key1, dtype: int64

In [111]:
# 약 30만건
new['ch_key2'].value_counts()

동물/식물    70294
동화       58538
외국어      31636
노래/율동    30097
숫자/계산    28588
만들기      28297
탈것/기계    27510
활동/운동    16917
친구/사람    14298
Name: ch_key2, dtype: int64

In [109]:
# 약 27만건만 ch_key3 적용됨
new['ch_key3'].value_counts()

동화       45898
외국어      36455
탈것/기계    35070
숫자/계산    32568
활동/운동    31011
동물/식물    29611
친구/사람    24576
만들기      23403
노래/율동    18252
Name: ch_key3, dtype: int64

In [94]:
new['pr_key1'].value_counts()

정서/사회성     75484
음악예술       71757
자연탐구       54860
과학기술       50789
언어논리       47001
활동/운동      37536
바른생활/안전    32025
수리논리       11095
Name: pr_key1, dtype: int64

In [95]:
new['genre_mid'].value_counts()

노래율동      125069
TV만화      103275
책          73869
놀이교실       33553
외국어        21261
시리즈         4983
애니          4921
액션/모험       3354
노래 율동       1689
창의학습        1553
예능          1331
어린이방송       1200
독서동화        1134
코미디          721
MCN          379
유아애니         362
드라마          348
육아정보         330
한글           274
수학과학         270
다큐멘터리        237
극장판 애니       213
역사/문화        169
체육안전          23
시사교양          22
한자             4
예술             2
연예오락           1
Name: genre_mid, dtype: int64

In [97]:
meta_df.drop_duplicates('album_id').genre_mid.value_counts()

TV만화      10536
놀이교실      10433
노래율동       7000
외국어        2606
애니         2062
예능         1719
책          1151
창의학습        790
액션/모험       696
유아애니        536
드라마         452
시리즈         380
독서동화        313
육아정보        258
시사교양        203
노래 율동       129
MCN          91
코미디          80
어린이방송        79
연예오락         62
수학과학         60
한글           60
다큐멘터리        44
한자           38
역사/문화        35
예술           21
휴먼/감동        20
체육안전         13
극장판 애니        8
Name: genre_mid, dtype: int64

In [101]:
genre_ch1 = pd.crosstab(new['genre_mid'], new['ch_key1']).style.background_gradient(cmap='summer_r')
genre_ch1

ch_key1,노래/율동,동물/식물,동화,만들기,숫자/계산,외국어,친구/사람,탈것/기계,활동/운동
genre_mid,,,,,,,,,
MCN,59,121,34,53,18,26,6,33,29
TV만화,47489,14514,9482,7552,4713,5693,2071,8348,3413
극장판 애니,83,47,25,11,10,10,5,12,10
노래 율동,918,263,105,87,77,111,11,58,59
노래율동,79638,13108,8051,5435,3427,4761,1378,6349,2922
놀이교실,12099,5761,3723,3603,1953,2525,503,2282,1104
다큐멘터리,74,54,18,17,28,19,5,11,11
독서동화,417,202,157,134,51,73,30,53,17
드라마,114,65,36,58,15,38,2,10,10


In [105]:
genre_pr1 = pd.crosstab(new['genre_mid'], new['pr_key1']).style.background_gradient(cmap='summer_r')
genre_pr1

pr_key1,과학기술,바른생활/안전,수리논리,언어논리,음악예술,자연탐구,정서/사회성,활동/운동
genre_mid,,,,,,,,
MCN,143,13,17,31,55,74,24,22
TV만화,12754,8915,2862,12006,18871,13794,21706,12367
극장판 애니,23,14,2,21,37,41,46,29
노래 율동,137,135,76,176,354,343,262,206
노래율동,9076,11987,2220,14489,26173,19564,27041,14519
놀이교실,5213,2620,1039,4306,5946,5869,5703,2857
다큐멘터리,49,6,7,40,35,47,42,11
독서동화,131,57,40,154,204,228,248,72
드라마,37,27,4,59,85,48,61,27


In [102]:
genre_ch2 = pd.crosstab(new['genre_mid'], new['ch_key2']).style.background_gradient(cmap='summer_r')
genre_ch2

ch_key2,노래/율동,동물/식물,동화,만들기,숫자/계산,외국어,친구/사람,탈것/기계,활동/운동
genre_mid,,,,,,,,,
MCN,24,54,45,38,44,22,6,13,71
TV만화,9246,18204,16563,6869,7254,7817,3789,7867,4889
극장판 애니,20,47,32,11,12,12,11,18,9
노래 율동,126,443,210,100,96,122,57,147,109
노래율동,9542,31319,17422,4807,6573,9384,3817,9399,5027
놀이교실,2591,4917,4616,3675,2920,3130,1665,2493,1743
다큐멘터리,14,39,26,35,17,10,14,19,8
독서동화,171,207,138,94,61,77,85,65,34
드라마,33,61,42,45,19,50,34,11,12


In [103]:
genre_ch3 = pd.crosstab(new['genre_mid'], new['ch_key3']).style.background_gradient(cmap='summer_r')
genre_ch3

ch_key3,노래/율동,동물/식물,동화,만들기,숫자/계산,외국어,친구/사람,탈것/기계,활동/운동
genre_mid,,,,,,,,,
MCN,13,16,38,44,18,33,33,10,34
TV만화,4729,8414,11528,5920,8386,8694,6623,10567,8390
극장판 애니,17,17,25,10,16,16,13,24,12
노래 율동,66,167,189,84,135,141,98,238,157
노래율동,5198,10421,14836,5082,8721,10721,8632,12951,9954
놀이교실,1867,2466,3491,2904,3175,3439,1941,3133,3166
다큐멘터리,12,17,26,10,27,29,10,28,13
독서동화,62,73,208,66,82,120,80,74,111
드라마,29,11,70,26,23,8,81,22,34


# 행동점수

In [6]:
behav_df = pd.read_csv('./behavior.csv', encoding='utf-8')

In [96]:
behav_df

,profile_id,album_id,start,yes_watch,buy,search
0,3,15,1.0,1.0,0.0,0.0
1,3,16,1.0,1.0,0.0,0.0
2,3,17,1.0,1.0,0.0,0.0
3,3,18,1.0,1.0,0.0,0.0
4,3,19,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...
383809,32193,5391,0.0,0.0,0.0,1.0
383810,32193,2271,0.0,0.0,0.0,1.0
383811,32193,26071,0.0,0.0,0.0,1.0
383812,32193,551,0.0,0.0,0.0,1.0


In [97]:
# 시청시작 데이터는 없는데 시청시작은 있는 경우
behav_df.loc[(behav_df['start'] == 0) & (behav_df['yes_watch'] == 1)]

# 무려 2308건

,profile_id,album_id,start,yes_watch,buy,search
380548,30,3669,0.0,1.0,0.0,0.0
380549,96,849,0.0,1.0,0.0,0.0
380550,179,3168,0.0,1.0,0.0,0.0
380553,192,4324,0.0,1.0,0.0,0.0
380554,192,6978,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...
383713,33016,4456,0.0,1.0,0.0,0.0
383716,33032,709,0.0,1.0,0.0,0.0
383726,33032,8838,0.0,1.0,0.0,0.0
383728,33032,2435,0.0,1.0,0.0,0.0


In [113]:
behav_df.loc[(behav_df['start'] == 0) & (behav_df['buy'] == 1)]

,profile_id,album_id,start,yes_watch,buy,search


In [119]:
behav_df.loc[(behav_df['start'] == 0) & (behav_df['search'] == 1)]
# 검색하여 시청에는 있는데, 시청 시작에는 없는 79개 행

,profile_id,album_id,start,yes_watch,buy,search
383735,551,782,0.0,0.0,0.0,1.0
383736,1639,5855,0.0,0.0,0.0,1.0
383737,1835,3838,0.0,0.0,0.0,1.0
383738,2629,9306,0.0,0.0,0.0,1.0
383739,3451,4054,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...
383809,32193,5391,0.0,0.0,0.0,1.0
383810,32193,2271,0.0,0.0,0.0,1.0
383811,32193,26071,0.0,0.0,0.0,1.0
383812,32193,551,0.0,0.0,0.0,1.0


In [116]:
behav_df.loc[(behav_df['buy'] == 1) & (behav_df['search'] == 1)]

,profile_id,album_id,start,yes_watch,buy,search
5889,411,5612,1.0,1.0,1.0,1.0
5894,411,5617,1.0,1.0,1.0,1.0
5914,411,5634,1.0,1.0,1.0,1.0
5917,411,5637,1.0,1.0,1.0,1.0
5918,411,5638,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...
355665,30766,16106,1.0,0.0,1.0,1.0
370196,32012,673,1.0,1.0,1.0,1.0
372277,32184,20933,1.0,0.0,1.0,1.0
372279,32184,9589,1.0,0.0,1.0,1.0


In [121]:
behav_df.loc[(behav_df['search'] == 1) & (behav_df['buy'] == 1)]

,profile_id,album_id,start,yes_watch,buy,search
5889,411,5612,1.0,1.0,1.0,1.0
5894,411,5617,1.0,1.0,1.0,1.0
5914,411,5634,1.0,1.0,1.0,1.0
5917,411,5637,1.0,1.0,1.0,1.0
5918,411,5638,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...
355665,30766,16106,1.0,0.0,1.0,1.0
370196,32012,673,1.0,1.0,1.0,1.0
372277,32184,20933,1.0,0.0,1.0,1.0
372279,32184,9589,1.0,0.0,1.0,1.0


In [100]:
history_df.loc[(history_df['profile_id'] == 30) & (history_df['album_id'] == 3669)]

,profile_id,ss_id,log_time,act_target_dtl,album_id,payment,continuous_play,short_trailer


In [101]:
we_df.loc[(we_df['profile_id'] == 30) & (we_df['album_id'] == 3669)]

,profile_id,ss_id,log_time,act_target_dtl,album_id,watch_time,total_time,continuous_play
2866,30,20220408224805,20220408225585,MKID049,3669,184,186,1


In [105]:
behav_df.loc[behav_df['search'] == 1]
# 3613건 뿐

,profile_id,album_id,start,yes_watch,buy,search
1194,59,2141,1.0,0.0,0.0,1.0
1195,59,512,1.0,1.0,0.0,1.0
1196,59,2142,1.0,0.0,0.0,1.0
1197,59,2143,1.0,1.0,0.0,1.0
1204,59,2150,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...
383809,32193,5391,0.0,0.0,0.0,1.0
383810,32193,2271,0.0,0.0,0.0,1.0
383811,32193,26071,0.0,0.0,0.0,1.0
383812,32193,551,0.0,0.0,0.0,1.0


In [106]:
behav_df.loc[behav_df['buy'] == 1]

,profile_id,album_id,start,yes_watch,buy,search
38,5,84,1.0,1.0,1.0,0.0
134,5,178,1.0,0.0,1.0,0.0
137,5,181,1.0,0.0,1.0,0.0
203,5,233,1.0,1.0,1.0,0.0
204,5,234,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...
380529,33032,349,1.0,0.0,1.0,0.0
380530,33032,350,1.0,1.0,1.0,0.0
380532,33032,466,1.0,0.0,1.0,0.0
380537,33032,673,1.0,0.0,1.0,0.0


In [107]:
search_df

,profile_id,ss_id,log_time,act_target_dtl,album_id
0,59,20220303183951,20220303183914,MKID017,2141
1,59,20220303183951,20220303183914,MKID017,2141
2,59,20220303183951,20220303183914,MKID017,2141
3,59,20220304181433,20220304181525,MKID017,512
4,59,20220304181433,20220304182447,MKID017,2141
...,...,...,...,...,...
6469,32965,20220423161110,20220423164503,MKID017,23083
6470,32965,20220424065420,20220424065456,MKID017,15080
6471,32965,20220425073255,20220425073358,MKID017,15080
6472,32965,20220427062749,20220427063353,MKID017,16497


# make each profile_id's "continuous_play_ratio" variable

In [7]:
history_df

,profile_id,ss_id,log_time,act_target_dtl,album_id,payment,continuous_play,short_trailer
0,3,20220301115653,20220301115719,MKID003,15,NaN,Y,N
1,3,20220301115653,20220301115809,MKID003,16,NaN,Y,N
2,3,20220301115653,20220301115958,MKID003,17,NaN,Y,N
3,3,20220301115653,20220301120118,MKID003,18,NaN,Y,N
4,3,20220301115653,20220301120229,MKID003,19,NaN,Y,N
...,...,...,...,...,...,...,...,...
1005646,33032,20220427155091,20220427155668,MKID003,381,NaN,Y,N
1005647,33032,20220427155091,20220427155680,MKID003,381,NaN,Y,N
1005648,33032,20220427155839,20220427155810,MKID003,125,NaN,Y,N
1005649,33032,20220427155706,20220427155838,MKID003,125,NaN,Y,N


In [17]:
new_history = history_df

In [18]:
le = LabelEncoder()
new_history['continuous_play'] = le.fit_transform(new_history['continuous_play'])

In [19]:
new_history

,profile_id,ss_id,log_time,act_target_dtl,album_id,payment,continuous_play,short_trailer
0,3,20220301115653,20220301115719,MKID003,15,NaN,1,N
1,3,20220301115653,20220301115809,MKID003,16,NaN,1,N
2,3,20220301115653,20220301115958,MKID003,17,NaN,1,N
3,3,20220301115653,20220301120118,MKID003,18,NaN,1,N
4,3,20220301115653,20220301120229,MKID003,19,NaN,1,N
...,...,...,...,...,...,...,...,...
1005646,33032,20220427155091,20220427155668,MKID003,381,NaN,1,N
1005647,33032,20220427155091,20220427155680,MKID003,381,NaN,1,N
1005648,33032,20220427155839,20220427155810,MKID003,125,NaN,1,N
1005649,33032,20220427155706,20220427155838,MKID003,125,NaN,1,N


In [1]:
test = {'user_index_1':{'item_index_1': 1}}

In [4]:
test['user_index_1']['item_index_1']

1

In [97]:
new_history.loc[(new_history['profile_id'] == 33032) & (new_history['album_id'] == 8910)]

,profile_id,ss_id,log_time,act_target_dtl,album_id,payment,continuous_play,short_trailer
1005526,33032,20220426115550,20220426130410,MKID003,8910,NaN,0,N


In [21]:
new_history.continuous_play.cumsum()

0               1
1               2
2               3
3               4
4               5
            ...  
1005646    889514
1005647    889515
1005648    889516
1005649    889517
1005650    889517
Name: continuous_play, Length: 1005651, dtype: int64

In [20]:
# continuous_play 기준으로 사용자의 각 아이템에 대한 연속 재생 횟수 누적합을 나타냄
def cum_sum_conti_play(group):
    group['play_cumsum'] = group.sort_values(
        by=['profile_id', 'album_id'], ascending=False).continuous_play.cumsum()
    return group

In [22]:
hist_d = new_history[['profile_id', 'album_id', 'continuous_play']]

In [25]:
hist_d = hist_d.groupby(['profile_id', 'album_id']).apply(cum_sum_conti_play)

In [34]:
hist_d

,profile_id,album_id,continuous_play,play_cumsum
0,3,15,1,1
1,3,16,1,1
2,3,17,1,1
3,3,18,1,1
4,3,19,1,1
...,...,...,...,...
1005646,33032,381,1,6
1005647,33032,381,1,7
1005648,33032,125,1,5
1005649,33032,125,1,6


In [52]:
play_cumsum_max = pd.DataFrame(hist_d.groupby(['profile_id', 'album_id']).play_cumsum.max()).reset_index()

In [53]:
play_cumsum_max

,profile_id,album_id,play_cumsum
0,3,15,1
1,3,16,1
2,3,17,1
3,3,18,1
4,3,19,1
...,...,...,...
380542,33032,7105,0
380543,33032,8910,0
380544,33032,14662,1
380545,33032,14663,2


In [54]:
play_cumsum_max.columns = ['profile_id', 'album_id', 'conti_play_max']

In [55]:
play_cumsum_max

,profile_id,album_id,conti_play_max
0,3,15,1
1,3,16,1
2,3,17,1
3,3,18,1
4,3,19,1
...,...,...,...
380542,33032,7105,0
380543,33032,8910,0
380544,33032,14662,1
380545,33032,14663,2


In [95]:
hist_d.loc[(hist_d['profile_id'] == 33032) & (hist_d['album_id'] == 8910)]

,profile_id,album_id,continuous_play,play_cumsum
380543,33032,8910,1,1


In [28]:
hist_d['play_cumsum'].max()

1414

In [27]:
hist_d['play_cumsum'].describe()

count    1.005651e+06
mean     1.329027e+01
std      6.714049e+01
min      0.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      5.000000e+00
max      1.414000e+03
Name: play_cumsum, dtype: float64

In [85]:
hist_d = hist_d.sort_values(by=['profile_id', 'album_id'], ascending=False).groupby(['profile_id', 'album_id']).count().reset_index()

In [104]:
new_play = pd.concat([hist_d, play_cumsum_max['conti_play_max']], axis = 1)
new_play = new_play[['profile_id', 'album_id', 'play_cumsum', 'conti_play_max']]
new_play.columns = ['profile_id', 'album_id', 'play_count', 'conti_play_max']
new_play['conti_play_ratio'] = new_play['conti_play_max'] / new_play['play_count']
new_play['conti_play_ratio'] = new_play['conti_play_ratio'].fillna(0)
new_play

,profile_id,album_id,play_count,conti_play_max,conti_play_ratio
0,3,15,1,1,1.0
1,3,16,1,1,1.0
2,3,17,1,1,1.0
3,3,18,1,1,1.0
4,3,19,1,1,1.0
...,...,...,...,...,...
380542,33032,7105,21,0,0.0
380543,33032,8910,1,0,0.0
380544,33032,14662,1,1,1.0
380545,33032,14663,2,2,1.0


In [111]:
new_play.to_csv('conti_play_ratio.csv', index = False)

In [112]:
import pickle

with open('conti_play_ratio.pickle', 'wb') as f:
    pickle.dump(new_play, f)

In [105]:
new_play.conti_play_ratio.describe()

count    380547.000000
mean          0.926074
std           0.242106
min           0.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           1.000000
Name: conti_play_ratio, dtype: float64

In [108]:
new_play.loc[(new_play['conti_play_ratio'] != 1) & (new_play['conti_play_ratio'] != 0)].conti_play_ratio.describe()

count    18675.000000
mean         0.573906
std          0.209554
min          0.012195
25%          0.500000
50%          0.500000
75%          0.750000
max          0.997067
Name: conti_play_ratio, dtype: float64

In [110]:
history_df['continuous_play'].describe()

count    1.005651e+06
mean     8.845186e-01
std      3.196022e-01
min      0.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.000000e+00
Name: continuous_play, dtype: float64

## make 'watch' variable

In [21]:
### cumsum
# 어떤 데이터프레임에 새로운 칼럼 cumtime을 추가
# watch_time변수를 기준으로 사용자의 각 아이템에 대한 시청 시간 누적합을 나타냄
def cum_sum(group):
    group['cumtime'] = group.sort_values(
        by=['profile_id', 'album_id'], ascending=False).watch_time.cumsum()
    return group

In [22]:
# 시청 종료 데이터에서 실제 시청 길이인 'watch_time'과 콘텐츠 길이인'total_time' 칼럼 살림
we_d = we_df[["profile_id", "album_id", "act_target_dtl", "watch_time", "total_time"]]

# 누적 시청 시간인 cumtime 변수를 cum_sum 함수를 적용하여 생성
we_d = we_d.groupby(["profile_id","album_id"]).apply(cum_sum)

KeyboardInterrupt: 

In [ ]:
# 동일 콘텐츠의 재생 횟수 의미하는 'watch' 칼럼 추가
we_d["watch"] = we_d["cumtime"]/we_d["total_time"]

In [ ]:
watch = pd.DataFrame(we_d.groupby(['profile_id', 'album_id']).watch.max()).reset_index()

In [42]:
watch_dict = dict()
for index, profile, album, watch in watch.itertuples():
    watch_dict[(profile, album)] = watch

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [44]:
len(watch_dict)

347982

In [47]:
watch_dict

{(3, 15): 1.0,
 (3, 16): 0.9904761904761905,
 (3, 17): 1.0,
 (3, 18): 0.9852941176470589,
 (3, 19): 1.0,
 (3, 20): 2.0,
 (3, 21): 1.0756302521008403,
 (3, 22): 1.0,
 (3, 23): 0.9917355371900827,
 (3, 24): 1.0,
 (3, 25): 1.0,
 (3, 26): 1.0,
 (3, 27): 1.0,
 (3, 28): 0.9915966386554622,
 (3, 29): 1.0,
 (3, 30): 0.9917355371900827,
 (3, 31): 1.0,
 (3, 32): 1.0,
 (5, 0): 0.9926470588235294,
 (5, 15): 4.6521739130434785,
 (5, 16): 4.066666666666666,
 (5, 17): 3.0,
 (5, 18): 3.985294117647059,
 (5, 19): 3.0,
 (5, 38): 2.2378378378378376,
 (5, 39): 1.9945945945945946,
 (5, 40): 0.9945945945945946,
 (5, 41): 0.9944444444444445,
 (5, 42): 2.967741935483871,
 (5, 43): 0.9945945945945946,
 (5, 44): 1.0,
 (5, 45): 1.9891891891891893,
 (5, 46): 1.3297297297297297,
 (5, 50): 0.9928571428571429,
 (5, 51): 1.0,
 (5, 65): 0.5634328358208955,
 (5, 66): 0.043689320388349516,
 (5, 67): 1.1956521739130435,
 (5, 68): 0.3137254901960784,
 (5, 69): 1.0,
 (5, 70): 1.8992481203007519,
 (5, 71): 3.014084507042253

In [ ]:
we_d["yes_watch"] = 0
we_d["yes_watch"][we_d["watch"] >= 0.5] = 1

In [45]:
behav_df.yes_watch

0         1.0
1         1.0
2         1.0
3         1.0
4         1.0
         ... 
383809    0.0
383810    0.0
383811    0.0
383812    0.0
383813    0.0
Name: yes_watch, Length: 383814, dtype: float64

In [9]:
import pickle

In [10]:
with open("watching.pickle", 'rb') as f:
     data = pickle.load(f)

In [76]:
data.watch.quantile(0.991)

92.26275773195984

In [72]:
data

,profile_id,album_id,watch
0,3,15,1.000000
2,3,16,1.980952
3,3,17,1.000000
5,3,18,1.970588
6,3,19,1.000000
...,...,...,...
5785240,33032,3559,1.000000
5785244,33032,4246,0.269802
5785188,33032,6693,0.998498
5785418,33032,7105,11.211838


In [94]:
we_df.loc[(we_df['profile_id'] == 22) & (we_df['album_id'] == 593)]

,profile_id,ss_id,log_time,act_target_dtl,album_id,watch_time,total_time,continuous_play
1024,22,20220304214739,20220304215050,MKID049,593,137,137,1
1027,22,20220304214739,20220304220445,MKID049,593,137,139,1
1060,22,20220305213534,20220305221041,MKID049,593,137,137,1
1061,22,20220305213534,20220305221041,MKID049,593,137,137,1
1099,22,20220306091700,20220306093411,MKID049,593,137,139,1
1143,22,20220306215419,20220306223602,MKID049,593,71,137,1
1144,22,20220306215419,20220306223616,MKID049,593,5,137,0
1145,22,20220306215419,20220306223646,MKID049,593,22,137,0
1146,22,20220306215419,20220306223702,MKID049,593,9,137,0
1147,22,20220306215419,20220306223925,MKID049,593,137,137,0


In [92]:
data.loc[data['watch'] > 100][0:1]

,profile_id,album_id,watch
4615,22,593,114.949275


In [93]:
data2[(22, 593)]

16.766423357664234

In [82]:
meta_df.loc[meta_df['album_id'] == 3502]

,album_id,title,sub_title,genre_large,genre_mid,genre_small,country,run_time,onair_date,cast_1,cast_2,cast_3,cast_4,cast_5,cast_6,cast_7
5595,3502,콩순이 진찰송,콩순이의 율동교실1,키즈,노래율동,NaN,NaN,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
data.watch.sort_values(ascending = False)[:2000]

1162875    187876.850000
2092838     79232.214047
1162713     70098.692308
1163777     64549.228261
1180047     64353.202247
               ...      
4599340       132.837662
4350754       132.802676
205980        132.800000
149598        132.800000
5114079       132.774074
Name: watch, Length: 2000, dtype: float64

In [12]:
data.watch.describe()

count    328526.000000
mean         10.064806
std         483.915019
min           0.000000
25%           0.730489
50%           1.000000
75%           2.764085
max      187876.850000
Name: watch, dtype: float64

In [13]:
with open("watch.pickle", 'rb') as f:
     data2 = pickle.load(f)

In [64]:
len(data2.values())

347982

In [69]:
big_value_list = []
for i in data2.values():
    if i >= 50:
        big_value_list.append(i)

In [70]:
len(big_value_list)

375

In [68]:
big_value_list

[50.88721804511278,
 37.97011952191235,
 34.94897959183673,
 48.0722891566265,
 32.98715203426124,
 45.002164502164504,
 42.266666666666666,
 38.92599620493359,
 31.459627329192546,
 34.76712328767123,
 44.3394495412844,
 43.352112676056336,
 35.02173913043478,
 32.923809523809524,
 33.6578947368421,
 33.55555555555556,
 53.4375,
 64.9493670886076,
 63.85436893203884,
 52.56043956043956,
 57.03260869565217,
 58.05,
 154.86624203821657,
 47.56962025316456,
 31.95505617977528,
 69.96190476190476,
 45.87096774193548,
 222.66315789473686,
 76.8659793814433,
 35.58252427184466,
 83.03225806451613,
 94.3298969072165,
 96.62105263157895,
 62.8,
 35.329896907216494,
 31.723809523809525,
 33.76470588235294,
 31.338983050847457,
 35.47826086956522,
 31.961904761904762,
 32.97826086956522,
 43.095238095238095,
 32.0921052631579,
 32.38235294117647,
 44.205882352941174,
 42.47826086956522,
 56.59574468085106,
 37.2,
 75.625,
 30.186813186813186,
 37.39156626506024,
 34.77297297297297,
 32.82162162

In [60]:
sorted(value_list, reverse = True)

[2042.0375,
 1039.6818181818182,
 912.0957446808511,
 883.5822784810126,
 868.021978021978,
 516.080459770115,
 492.9837837837838,
 492.0,
 489.1142857142857,
 488.04761904761904,
 482.4742268041237,
 477.17894736842106,
 474.38829787234044,
 472.9578947368421,
 467.3440860215054,
 467.16494845360825,
 420.69565217391306,
 405.530303030303,
 335.3333333333333,
 300.7608695652174,
 270.601246105919,
 269.4891304347826,
 243.32608695652175,
 241.82608695652175,
 237.8877551020408,
 237.13622291021673,
 229.05295950155764,
 228.92834890965733,
 222.66315789473686,
 220.126213592233,
 216.40816326530611,
 212.125,
 210.19117647058823,
 209.32323232323233,
 197.23684210526315,
 195.6524064171123,
 193.13,
 190.71428571428572,
 188.74725274725276,
 183.60752688172042,
 182.13333333333333,
 176.84782608695653,
 176.2058823529412,
 175.4255319148936,
 174.96052631578948,
 168.5,
 166.71428571428572,
 166.00928792569658,
 164.58878504672896,
 162.21981424148606,
 154.86624203821657,
 154.088235

### 학습 및 검증 데이터 생성 
- train : 시청 이력의 80%를 사용 
- valid : 시청 이력의 20%를 사용 

In [52]:
# 데이터 전처리 (중복제거) 
# 참고 : drop_duplicates의 subset을 무엇으로 구성하냐에 따라서 제거되는 항목들이 다름 
# ex) 'profile_id', 'album_id' : 중복된 시청이력 모두 제거 / 'profile_id', 'album_id', 'log_time' : 같은 시간에 시청한 이력만 제거 
data = history_df[['profile_id', 'album_id']].drop_duplicates(subset=['profile_id', 'album_id']).sort_values(by = ['profile_id']).reset_index(drop = True)
data['rating'] = 1

cfg.n_users = data.profile_id.max()+1
cfg.n_items = data.album_id.max()+1

In [53]:
# 학습 및 검증 데이터 분리
train, valid = train_test_split(
    data, test_size=cfg.test_size, random_state=cfg.seed,
)
print('학습 데이터 크기:', train.shape)
print('검증 데이터 크기:', valid.shape)

학습 데이터 크기: (304437, 3)
검증 데이터 크기: (76110, 3)


In [54]:
# Matrix 형태로 변환 
train = train.to_numpy()
matrix = sparse.lil_matrix((cfg.n_users, cfg.n_items))  #
for (p, i, r) in tqdm(train):
    matrix[p, i] = r
    
train = sparse.csr_matrix(matrix)
train = train.toarray()
print("train 형태: \n", train)

  0%|          | 0/304437 [00:00<?, ?it/s]

train 형태: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [55]:
profile_df = profile_df.set_index('profile_id')

In [56]:
le = LabelEncoder()
profile_df['sex'] = le.fit_transform(profile_df['sex'])
profile_df['pr_interest_keyword_cd_1'] = le.fit_transform(profile_df['pr_interest_keyword_cd_1'])
profile_df['pr_interest_keyword_cd_2'] = le.fit_transform(profile_df['pr_interest_keyword_cd_2'])
profile_df['pr_interest_keyword_cd_3'] = le.fit_transform(profile_df['pr_interest_keyword_cd_3'])
profile_df['ch_interest_keyword_cd_1'] = le.fit_transform(profile_df['ch_interest_keyword_cd_1'])
profile_df['ch_interest_keyword_cd_2'] = le.fit_transform(profile_df['ch_interest_keyword_cd_2'])
profile_df['ch_interest_keyword_cd_3'] = le.fit_transform(profile_df['ch_interest_keyword_cd_3'])

In [57]:
# 유저 특징 정보 추출 
user_features = profile_df[['age', 'sex',
                            'pr_interest_keyword_cd_1',
                            'pr_interest_keyword_cd_2',
                            'pr_interest_keyword_cd_3',
                            'ch_interest_keyword_cd_1',
                            'ch_interest_keyword_cd_2',
                            'ch_interest_keyword_cd_3']].to_dict()
print("user_id 3의 age 정보 :", user_features['age'][3])

user_id 3의 age 정보 : 5


In [58]:
user_features['pr_interest_keyword_cd_1']

{3: 1,
 5: 6,
 7: 4,
 12: 2,
 16: 2,
 19: 4,
 20: 6,
 22: 5,
 24: 4,
 26: 0,
 27: 1,
 30: 0,
 31: 4,
 33: 1,
 35: 7,
 55: 5,
 56: 0,
 59: 2,
 62: 6,
 63: 0,
 66: 6,
 68: 1,
 74: 1,
 77: 5,
 80: 5,
 89: 6,
 94: 5,
 96: 5,
 101: 5,
 102: 2,
 103: 0,
 109: 1,
 110: 4,
 115: 5,
 116: 3,
 118: 0,
 121: 7,
 122: 4,
 124: 0,
 127: 6,
 129: 7,
 130: 6,
 136: 5,
 137: 5,
 139: 1,
 144: 1,
 150: 6,
 152: 0,
 156: 3,
 165: 5,
 170: 5,
 172: 0,
 176: 3,
 179: 1,
 181: 0,
 182: 4,
 187: 4,
 192: 0,
 193: 6,
 195: 1,
 198: 5,
 201: 4,
 207: 2,
 209: 6,
 211: 0,
 214: 1,
 220: 7,
 228: 0,
 229: 4,
 230: 3,
 233: 3,
 240: 5,
 248: 1,
 251: 1,
 255: 0,
 258: 5,
 260: 0,
 261: 0,
 264: 2,
 279: 6,
 290: 0,
 292: 6,
 293: 7,
 300: 4,
 302: 1,
 303: 5,
 304: 1,
 307: 5,
 309: 3,
 316: 5,
 317: 6,
 321: 5,
 326: 1,
 327: 5,
 335: 3,
 338: 6,
 345: 1,
 346: 0,
 349: 1,
 350: 4,
 358: 6,
 361: 5,
 363: 6,
 364: 4,
 369: 4,
 370: 7,
 372: 3,
 373: 0,
 384: 1,
 390: 2,
 395: 0,
 397: 7,
 399: 5,
 402: 6,
 407:

In [59]:
meta_df = meta_df.set_index('album_id')

In [60]:
meta_df['genre_large'] = le.fit_transform(meta_df['genre_large'])
meta_df['genre_mid'] = le.fit_transform(meta_df['genre_mid'])
meta_df['genre_small'] = le.fit_transform(meta_df['genre_small'])
meta_df['cast_1'] = le.fit_transform(meta_df['cast_1'])
meta_df['cast_2'] = le.fit_transform(meta_df['cast_2'])
meta_df['cast_3'] = le.fit_transform(meta_df['cast_3'])
meta_df['cast_4'] = le.fit_transform(meta_df['cast_4'])
meta_df['cast_5'] = le.fit_transform(meta_df['cast_5'])
meta_df['cast_6'] = le.fit_transform(meta_df['cast_6'])
meta_df['cast_7'] = le.fit_transform(meta_df['cast_7'])

In [61]:
# 아이템 특징 정보 추출 
# 범주형 데이터를 수치형 데이터로 변경 

item_features = meta_df[['genre_large', 'genre_mid', 'genre_small',
                         'cast_1', 'cast_2', 'cast_3', 
                         'cast_4', 'cast_5', 'cast_6', 'cast_7']].to_dict()


In [62]:
print("album_id 749의 genre_mid 정보 :", item_features['genre_small'][749])

album_id 749의 genre_mid 정보 : 20


In [59]:
meta_df['cast_1'].value_counts()

689    14999
279     1628
654     1201
566      996
530      792
       ...  
321        1
132        1
638        1
563        1
667        1
Name: cast_1, Length: 690, dtype: int64

In [56]:
meta_df['genre_small'].value_counts()

20    29183
10     4225
14     3848
6      1159
3      1027
13      989
5       554
11      504
8       280
18      229
2       215
0       212
16       68
7        41
19       24
15       13
4        10
9         9
17        7
12        4
1         1
Name: genre_small, dtype: int64